In [2]:
import atexit, threading, json
from http.server import BaseHTTPRequestHandler, HTTPServer
import urllib.parse
import onem2mlib
import onem2mlib.exceptions as EXC
import onem2mlib.constants as CON
import onem2mlib.internal as INT
import requests
import uuid

In [1]:
# Encryption
import re
import base64
from base64 import b64decode
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad

In [3]:
def decode_base64(data, altchars=b'+/'):
    data = re.sub(rb'[^a-zA-Z0-9%s]+' % altchars, b'', data)
    missing_padding = len(data) % 4
    if missing_padding:
        data += b'='* (4 - missing_padding)
    return base64.b64decode(data, altchars)

def decode_AES(ciphertext):
    key = "2B7E151628AED2A6ABF7158809CF4F3C"
    iv = "AAAAAAAAAAAAAAAAAAAAAA=="

    key = bytearray.fromhex(key)
    iv = b64decode(iv).hex()
    iv = bytearray.fromhex(iv)
    ciphertext = b64decode(ciphertext)

    cipher = AES.new(key, AES.MODE_CBC, iv)
    decrypted = cipher.decrypt(ciphertext)
    decryptedUnpad = unpad(decrypted, AES.block_size)
    return decode_base64(decryptedUnpad).decode("utf-8")

In [4]:
decode_AES('IeuAnZdknufs4a5NL09+u0fBi/66Ro7U+4vhyURAE7rZB1V6a2CMmZ3A6XsO8Akm')

'10000003,0,0,0,10-11-13-39-29'

In [5]:
def print_content(content):
    content = content.split(',')
    unique_id, feature1, feature2, feature3, time_stamp =  content[0], content[1], content[2], content[3], content[4]

    # df = pd.read_csv('dataset.csv')
    # display(df)
    data_point = {
        "unique_id": unique_id,
        "feature_1": feature1,
        "feature_2": feature2,
        "feature_3": feature3,
        "time_stamp": time_stamp
    }
    print(data_point)

In [7]:
_server = None
_thread = None
_port = 1401


def call_back(res):
    print(res.content)
    dec_content = decode_AES(res.content)
    print(dec_content)
    print_content(dec_content)

# Start the notification server in a background threa
def _startNotificationServer():
	global _server, _thread
	if _thread:
		return
	# Start processing requests in a separate thread.
	# Listen on any interface/IP address.
	_server = HTTPNotificationServer(('', _port), HTTPNotificationHandler)
	_thread = threading.Thread(target=_server.run)
	_thread.start()

# Stop the thread/notification server
@atexit.register
def _stopNotificationServer():
	global _server, _thread
	if not _server or not _thread:
		return
	# Shutdown server
	_server.shutdown()
	_thread.join()
	_server = None
	_thread = None

# This class implements the notification server that runs in the background.
class HTTPNotificationServer(HTTPServer):
	def run(self):
		try:
			self.serve_forever()
		finally:
			# Clean-up server (close socket, etc.)
			self.server_close()

# This class implements the handler that reseives the requests
class HTTPNotificationHandler(BaseHTTPRequestHandler):
	# Handle incoming notifications (POST requests)
	def do_POST(self):
			# Construct return header
			self.send_response(200)
			self.send_header('X-M2M-RSC', '2000')
			self.end_headers()

			# Get headers and content data
			length = int(self.headers['Content-Length'])
			contentType = self.headers['Content-Type']
			post_data = self.rfile.read(length)
			#print(post_data)
			threading.Thread(target=self._handleJSON(post_data), args=(post_data)).start()
			

	# Catch and ignore all log messages
	def log_message(self, format, *args):
		return
	

	# Handle JSON notifications 
	def _handleJSON(self, data):
		jsn =  json.loads(data.decode('utf-8'))
		#print(jsn)
		# check verification request
		vrq = INT.getALLSubElementsJSON(jsn, 'vrq')
		if len(vrq) == 0:										
			vrq = INT.getALLSubElementsJSON(jsn, 'm2m:vrq')
		if len(vrq) > 0 and vrq[0] == True:
			return 	# do nothing

		# get the sur first
		sur = INT.getALLSubElementsJSON(jsn, 'sur')
		if len(sur) == 0:										
			sur = INT.getALLSubElementsJSON(jsn, 'm2m:sur')
		if len(sur) > 0:
			sur = sur[0]
		else:
			return 	# must have a subscription ID

		# get resource
		rep = INT.getALLSubElementsJSON(jsn, 'rep')
		if len(rep) == 0:							
			rep = INT.getALLSubElementsJSON(jsn, 'm2m:rep')
		if len(rep) > 0:
			jsn = rep[0]
			type = INT.getALLSubElementsJSON(jsn, 'ty')
			if type and len(type) > 0:
				resource = INT._newResourceFromType(type[0], None)
				resource._parseJSON(jsn)
				call_back(resource)
###############################################################################

In [8]:
_startNotificationServer()

In [9]:
def subscribe():
    global _port
    headers = {
        'X-M2M-Origin': 'rTI9uxe@NN:7rn5sOIaYf',
        'Content-type': 'application/json;ty=23'
     }
    
    body = {
        "m2m:sub":{
            "rn":str(uuid.uuid4()),
            "nu":[ f"http://10.11.0.194:{_port}/" ],
            "nct":1
        }
    }
    try:
        response = requests.post("http://esw-onem2m.iiit.ac.in:443/~/in-cse/in-name/Team-9/testing/", json=body, headers=headers)
    except TypeError:
        response = requests.post("http://esw-onem2m.iiit.ac.in:443/~/in-cse/in-name/Team-9/testing/", data=json.dumps(body), headers=headers)
    print('Return code : {}'.format(response.status_code))
    print('Return Content : {}'.format(response.text))
    return
    

In [10]:
subscribe()

Return code : 201
Return Content : {
   "m2m:sub" : {
      "rn" : "e101d304-1e99-48ac-97de-094782728940",
      "ty" : 23,
      "ri" : "/in-cse/sub-266087278",
      "pi" : "/in-cse/cnt-234319241",
      "ct" : "20211111T141007",
      "lt" : "20211111T141007",
      "acpi" : [ "/in-cse/acp-516227225", "/in-cse/acp-648169147" ],
      "nu" : [ "http://10.11.0.194:1401/" ],
      "nct" : 1
   }
}


In [47]:
_stopNotificationServer()

In [48]:
def get_data():
    headers = {
        'X-M2M-Origin': 'rTI9uxe@NN:7rn5sOIaYf',
        'Content-type': 'application/json'
     }

    response = requests.get("http://esw-onem2m.iiit.ac.in:443/~/in-cse/in-name/Team-9/testing?rcn=4", headers=headers)
    print('Return code : {}'.format(response.status_code))
    print('Return Content : {}'.format(response.text))
    _resp = json.loads(response.text)
    return response.status_code , _resp

In [49]:
i , j = get_data()

Return code : 200
Return Content : {
   "m2m:cnt" : {
      "rn" : "pir",
      "ty" : 3,
      "ri" : "/in-cse/cnt-423743632",
      "pi" : "/in-cse/CAE291430963",
      "ct" : "20211108T095038",
      "lt" : "20211108T095038",
      "acpi" : [ "/in-cse/acp-516227225", "/in-cse/acp-648169147" ],
      "et" : "20221108T095038",
      "st" : 3766,
      "mni" : 10,
      "mbs" : 10000,
      "mia" : 0,
      "cni" : 10,
      "cbs" : 10,
      "m2m:cin" : [ {
         "rn" : "cin_195969569",
         "ty" : 4,
         "ri" : "/in-cse/cin-195969569",
         "pi" : "/in-cse/cnt-423743632",
         "ct" : "20211109T004708",
         "lt" : "20211109T004708",
         "st" : 0,
         "cnf" : "text/plain:0",
         "cs" : 1,
         "con" : "0"
      }, {
         "rn" : "cin_722283552",
         "ty" : 4,
         "ri" : "/in-cse/cin-722283552",
         "pi" : "/in-cse/cnt-423743632",
         "ct" : "20211109T004710",
         "lt" : "20211109T004710",
         "st" : 0,
       